Import libaries

In [1]:
import numpy as np
from collections import Counter, OrderedDict
import collections
import pandas as pd

Read File

In [2]:
def readFile(filePath): 
    with open (filePath, 'r') as f: 
        lines = f.readlines()
        text = []
        label = []
        label_split = []
        temp_arr = []
        label_split_temp = []
        for l in lines: 
            if l != "\n":
                l_split = l.strip().split(" ")
                te = " ".join(l_split[:-1])
                la = l_split[-1]
                temp_arr.append((te, la))
                label.append(l_split[-1])
                label_split_temp.append(l_split[-1])
            else: 
                text.append(temp_arr)
                temp_arr = []
                label_split.append(label_split_temp)
                label_split_temp = []
    return text, label, label_split 

Calculate Transition Table

In [3]:
def transition_table(text, tra_arr):
    for ct, t in enumerate(text): 
        for ci, i in enumerate(t):
            if ci == len(t)-1:
                tra_psn = list(final_dict.keys()).index(i[1])
                tra_arr[tra_psn][len(final_dict)-1] += 1/final_dict[i[1]]
            else:
                if ci == 0: 
                    tra_psn = list(final_dict.keys()).index(i[1])
                    tra_arr[0][tra_psn-1] += 1/final_dict['START']
                cur_psn = list(final_dict.keys()).index(i[1])  
                next_psn = list(final_dict.keys()).index(t[ci+1][1])
                tra_arr[cur_psn][next_psn-1] += 1/final_dict[i[1]]
    return tra_arr
    

Preprocessing

In [10]:
def preprocess(filepath, lan):
    
    em_arr = pd.read_csv(filepath, sep=',',header=None)
    em_arr = em_arr.T
    em_arr = em_arr.drop([0, 2])
    em_arr.columns = em_arr.iloc[0]
    em_arr = em_arr[1:]
    if lan == 'EN':
        em_arr = em_arr.reindex([5,6,11,12,4,7,9,3,16,10,8,19,20,18,21,17,13,22,23,14,15])
    elif lan == 'CN':
        em_arr = em_arr.reindex([4,6,3,7,8,5,9])
    else: 
         em_arr = em_arr.reindex([6,3,4,7,5,9,8])
    header_list = list(em_arr.columns.values)
    em = em_arr.drop(['Text'], axis=1).values
    em = em.astype('float64')
    header_list = header_list[1:]
    return em, header_list

Get label index

In [5]:
def get_label_index(filepath, header_list):
    test = []
    output_index = []
    output_label = []
    index = []
    label = []
    with open(filepath, 'r') as f:
        lines = f.readlines()
        for l in lines: 
            if l != "\n":
                label.append(l.strip())
                try: 
                    index.append(header_list.index(l.strip()))
                except: 
                    index.append(header_list.index("#UNK#"))
            else: 
                output_index.append(index)
                output_label.append(label)
                index = []
                label = []
    return output_index, output_label

Get Label Dictionary

In [6]:
def get_label_dict(text, label): 
    final_dict = {'START': len(text)}
    label_counter = dict(Counter(label))
    final_dict.update(label_counter)
    return final_dict 

Get Transition Array

In [7]:
def get_tra_array(final_dict):
    tra_arr = np.zeros((len(final_dict),len(final_dict)))
    tra_arr = transition_table(text, tra_arr)
    return tra_arr

Vertibi Algo

In [14]:
from math import isclose

def vertibi_algo(tp, ep, d, n):

    # get values for first layer 
    first_arr = []
    for i in range(len(tp)-1):
        first_arr.append(tp[0][i] * ep[i][n[0]])

    # moving forward
    pi_arr = [] # store overall array
    dy_arr = [] # store array for each layer
    for c, i in enumerate(n):
        if c == 0:
            pass
        # second layer, use values from first layer
        elif c == 1: 
            for j in range(len(ep)): # the state
                temp_arr = []
                temp_dict = {}
                for q in range(len(ep)): # the previous state
                    temp_arr.append(first_arr[q] * tp[q+1][j] * ep[j][i])
                max_val = max(temp_arr)
                prev_max = list(d.keys())[temp_arr.index(max(temp_arr))+1]
                temp_dict[prev_max] = max_val
                temp_arr[temp_arr.index(max(temp_arr))] = -1
                max_val = max(temp_arr)
                prev_max = list(d.keys())[temp_arr.index(max(temp_arr))+1]
                temp_dict[prev_max] = max_val
                temp_arr[temp_arr.index(max(temp_arr))] = -1
                max_val = max(temp_arr)
                prev_max = list(d.keys())[temp_arr.index(max(temp_arr))+1]
                temp_dict[prev_max] = max_val
                dy_arr.append(temp_dict)
            pi_arr.append(dy_arr)
        # third layer onwards
        else:
            layer_arr = []
            for j in range(len(ep)): # the state
                temp_arr = []
                temp_dict = {}
                # dy stores list from previous layer
                for q in range(len(ep)): # the previous state
                    for w in range(3): # the three values in the previous state
                        val = list(dy_arr[q].values())[w] * tp[q+1][j] * ep[j][i]
                        temp_arr.append(val)
                max_val = max(temp_arr)
                prev_max = list(d.keys())[temp_arr.index(max(temp_arr))//3+1]
                temp_dict[prev_max] = max_val
                temp_arr[temp_arr.index(max(temp_arr))] = -1
                max_val = max(temp_arr)
                prev_max = list(d.keys())[temp_arr.index(max(temp_arr))//3+1]
                if prev_max in temp_dict:
                    temp_dict[prev_max+'1'] = max_val
                else: 
                    temp_dict[prev_max] = max_val
                temp_arr[temp_arr.index(max(temp_arr))] = -1
                max_val = max(temp_arr)
                prev_max = list(d.keys())[temp_arr.index(max(temp_arr))//3+1]
                if prev_max in temp_dict:
                    temp_dict[prev_max+'2'] = max_val
                else: 
                    temp_dict[prev_max] = max_val
                layer_arr.append(temp_dict)
            dy_arr = layer_arr
            pi_arr.append(layer_arr)
    
    # stop layer
    last_arr = []
    for j in range(len(ep)): # the previous state
        for q in range(3): # 2 values in each state
            last_arr.append(list(dy_arr[j].values())[q] * tp[j+1][-1])
    last_arr[last_arr.index(max(last_arr))] = -1
    last_arr[last_arr.index(max(last_arr))] = -1
    last_stateidx = last_arr.index(max(last_arr)) // 3 #0
    last_state = list(d.keys())[last_stateidx+1]
    seclast_stateidx = last_arr.index(max(last_arr)) % 3

    rst = []
    rst.append(list(d.keys())[last_stateidx+1])
    prev_state = list(pi_arr[-1][last_stateidx].keys())[seclast_stateidx]
    last_val = list(pi_arr[-1][last_stateidx].values())[seclast_stateidx]
    if prev_state[-1] == '1' or prev_state[-1] == '2':
            prev_state = prev_state[:-1]
    rst.append(prev_state)
    
    tempprev_state = ''
    prev_val = 0
    for i in range(len(n)-2):
        pi_arr.pop(-1)
        prev_dict = pi_arr[-1][list(d.keys()).index(prev_state)-1]
        if (tp[list(d.keys()).index(prev_state)][list(d.keys()).index(last_state)-1] * ep[list(d.keys()).index(last_state)-1][n[-1-i]]) != 0:
            prev_val = last_val / (tp[list(d.keys()).index(prev_state)][list(d.keys()).index(last_state)-1] * ep[list(d.keys()).index(last_state)-1][n[-1-i]])
        for j in range(len(prev_dict)):
            if isclose(prev_val, list(prev_dict.values())[j], rel_tol=1e-5, abs_tol=0.0):
                tempprev_state = list(prev_dict.keys())[j]
        if tempprev_state[-1] == '1' or tempprev_state[-1] == '2':
            tempprev_state = tempprev_state[:-1]
        last_val = prev_val
        last_state = prev_state
        prev_state = tempprev_state
        rst.append(prev_state)
        


    return(list(reversed(rst)))

Get output (Change file output name here)

In [15]:
def get_output(tra_arr, em, final_dict, index, label):
    seq = []
    for ci, i in enumerate(index): 
        seq_temp = vertibi_algo(tra_arr, em, final_dict, i)
        temp_arr = []
        for cj, j in enumerate(seq_temp): 
            temp_arr.append("{} {}".format(label[ci][cj],j))
        seq.append(temp_arr)

    with open ('dev.p4.out', 'w') as f: 
        for i in seq: 
            for j in i: 
                f.write(j+"\n")
            f.write("\n")
    return "Done"

In [ ]:
# Change File Path Here 
pre_filepath = '/Users/nicole/Desktop/CN/CN.csv'
devin_filepath = '/Users/nicole/Desktop/CN/dev.in'
train_filepath = '/Users/nicole/Desktop/CN/train'

text, label, label_split = readFile(train_filepath)
final_dict = get_label_dict(text, label)
tra_arr = get_tra_array(final_dict)
em, header_list = preprocess(pre_filepath, 'CN') # Indicate if it is EN/CN/SG
index, output_label = get_label_index(devin_filepath, header_list) #dev in


# Change output file name in get_output function
print(get_output(tra_arr, em, final_dict, index, output_label))